<a href="https://colab.research.google.com/github/High-Bee/honeycomb/blob/master/F_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-gpu==1.15


In [1]:
import tensorflow as tf
tf.__version__

'1.15.0'

In [5]:
# # Kaggle Fashion MNIST deeplearning
# # 필요한 module import

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from google.colab import drive
drive.mount('/content/drive')

# # Data Loading
k_train_mnist = pd.read_csv("/content/fashion-mnist_train.csv")
k_test_mnist = pd.read_csv("/content/fashion-mnist_test.csv")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
scaler = MinMaxScaler()
scaler.fit(k_train_mnist)

x_data = scaler.fit_transform((k_train_mnist.iloc[:,1:]/255))
y_data = pd.get_dummies(k_train_mnist["label"]).values

pre_x_data = scaler.fit_transform(k_test_mnist.values)


# train / test 구분
train_num = int(x_data.shape[0] * 0.8)

# train data set
train_x_data = x_data[:train_num]
train_y_data = y_data[:train_num]

# test data set
test_x_data = x_data[train_num:]
test_y_data = y_data[train_num:]

# Placeholder
tf.reset_default_graph()
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
drop_rate = tf.placeholder(dtype=tf.float32)

# Convolution Layer (Layer1)
x_img = tf.reshape(X, [-1,28,28,1]) # 몇장인지 모르고 28,28이고 색은 그레이스케일 1
W1 = tf.Variable(tf.random_normal([3,3,1,32]))
L1 = tf.nn.conv2d(x_img, W1, strides=[1,1,1,1], padding="SAME") # strides와 padding은 parameter값
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

# Layer2
W2 = tf.Variable(tf.random_normal([3,3,32,64]))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding="SAME")
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")



# 이렇게 만든 data를 FC Layer에 넣어서 학습!
L2 = tf.reshape(L2, [-1, 7*7*64])
#L2.shape # TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(64)])

W3 = tf.get_variable("weight3", shape=[7*7*64,256],
                        initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([256]), name="bias3")
_L3 = tf.nn.relu(tf.matmul(L2, W3)+b3)
L3 = tf.nn.dropout(_L3, rate=drop_rate)

# Layer 4
W4 = tf.get_variable("weight4", shape=[256,256],
                        initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([256]), name="bias4")
_L4 = tf.nn.relu(tf.matmul(L3, W4)+b4)
L4 = tf.nn.dropout(_L4, rate=drop_rate)

# Layer 5
W5 = tf.get_variable("weight5", shape=[256,10],
                        initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]), name="bias5")

# Hypothesis
logit = tf.matmul(L4, W5)+b5
H = tf.nn.relu(logit)

# Cost Function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logit,
                                                                    labels = Y))

# train
train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

# Session & reset
sess = tf.Session()
sess.run(tf.global_variables_initializer())


In [34]:
NL2.shape

TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(32)])

In [51]:
%%time
# 학습시켜보기! 
# batch function
train_epoch = 100
batch_size = 100


for step in range(train_epoch):
    num_of_iter = int(train_x_data.shape[0]/batch_size)
    cost_val = 0
    for i in range(num_of_iter):
        start = i * batch_size
        end = start + batch_size
        cut_train_x = train_x_data[start:end]
        cut_train_y = train_y_data[start:end]
        _, cost_val = sess.run([train, cost], 
                               feed_dict={ X : cut_train_x,
                                            Y : cut_train_y,
                                             drop_rate : 0.3})
    
        
    if step % 10 == 0:
        print("Cost값 : {}".format(cost_val))
      

Cost값 : 0.7583751082420349
Cost값 : 0.4109475016593933
Cost값 : 0.23193584382534027
Cost값 : 0.36803120374679565
Cost값 : 0.19683937728405
Cost값 : 0.23883932828903198
Cost값 : 0.36474645137786865
Cost값 : 0.2623209059238434
Cost값 : 0.1684090942144394
Cost값 : 0.3586425483226776
CPU times: user 1min 25s, sys: 13.4 s, total: 1min 38s
Wall time: 1min 28s


In [52]:
# 학습이 종료 되었으니 정확도를 측정

predict = tf.argmax(H, 1)
correct = tf.equal(predict, tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype = tf.float32))
print("정확도는 : {}".format(sess.run(accuracy, feed_dict={X : test_x_data,
                                                          Y : test_y_data,
                                                           drop_rate : 0.2})))


정확도는 : 0.8639794588088989
